In [20]:
import torch
import torch.nn as nn
from torchvision import models, transforms
import cv2
from PIL import Image
# Inicializar el modelo
model = models.resnet18(pretrained=False)
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
model.fc = nn.Linear(model.fc.in_features, 7)

# Cargar los pesos del modelo en la CPU
state_dict = torch.load('modelo_emociones_imagen.pth', map_location=torch.device('cpu'))

# Cargar los pesos en el modelo
model.load_state_dict(state_dict)

# Asegurarse de que el modelo esté en la CPU
device = torch.device('cpu')
model = model.to(device)

# Ahora el modelo está listo para usarse en la CPU


/var/folders/pv/hj1tkdtn4xd2lk842kp9xy7r0000gn/T/ipykernel_5972/2622852045.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load('modelo_emociones_ima

In [21]:
# Define las transformaciones necesarias para las imágenes
from torchvision import models, transforms

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((48, 48)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Mapeo de índices a nombres de emociones (esto debería coincidir con las etiquetas de tu dataset)
emotion_map = {0: 'Enojo', 1: 'Disgusto', 2: 'Miedo', 3: 'Felicidad', 4: 'Tristeza', 5: 'Sorpresa', 6: 'Neutral'}


In [24]:
import os

# Crear una carpeta para guardar las imágenes si no existe
output_dir = "captured_images"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Captura de video desde la cámara
cap = cv2.VideoCapture(0)  # 0 para la cámara por defecto

image_count = 0  # Contador para guardar imágenes con nombres únicos

while True:
    ret, frame = cap.read()  # Captura el frame actual
    if not ret:
        break

    # Guardar la imagen original capturada
    cv2.imwrite(f"{output_dir}/original_{image_count}.jpg", frame)

    # Preprocesa la imagen
    img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))  # Convertir a PIL Image
    img_tensor = transform(img).unsqueeze(0).to(device)  # Aplicar transformaciones y añadir batch dimension

    # Guardar la imagen preprocesada (escala de grises, redimensionada)
    img_gray = img_tensor.squeeze().cpu().numpy()  # Quitar la dimensión extra y convertir a numpy
    img_gray = (img_gray * 255).astype("uint8")  # Desnormalizar para guardar como imagen
    cv2.imwrite(f"{output_dir}/preprocessed_{image_count}.jpg", img_gray)

    # Realiza la predicción
    with torch.no_grad():
        output = model(img_tensor)
        _, predicted = torch.max(output, 1)
        emotion = emotion_map[predicted.item()]

    # Muestra la predicción en el frame
    cv2.putText(frame, f'Emocion: {emotion}', (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # Muestra el frame con la predicción
    cv2.imshow('Reconocimiento de Emociones en Tiempo Real', frame)

    # Incrementar el contador de imágenes
    image_count += 1

    # Presiona 'q' para salir del bucle
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Libera los recursos
cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [31]:
import torch
import torch.nn as nn
from torchvision import models, transforms
import cv2
from PIL import Image
import numpy as np
import os

# Inicializar el modelo
model = models.resnet18(pretrained=False)
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
model.fc = nn.Linear(model.fc.in_features, 7)

# Cargar los pesos del modelo en la CPU
state_dict = torch.load('modelo_emociones_imagen.pth', map_location=torch.device('cpu'))
model.load_state_dict(state_dict)

# Asegurarse de que el modelo esté en modo de evaluación y en la CPU
device = torch.device('cpu')
model = model.to(device)
model.eval()  # Asegurarse de que el modelo está en modo de evaluación

# Define las transformaciones necesarias, directamente redimensionando a 48x48
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((48, 48)),  # Redimensionar directamente a 48x48 para la predicción
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Mapeo de índices a nombres de emociones
emotion_map = {0: 'Enojo', 1: 'Disgusto', 2: 'Miedo', 3: 'Felicidad', 4: 'Tristeza', 5: 'Sorpresa', 6: 'Neutral'}

# Crear una carpeta para guardar las imágenes si no existe
output_dir = "captured_images"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Captura de video desde la cámara
cap = cv2.VideoCapture(0)  # 0 para la cámara por defecto

image_count = 0  # Contador para guardar imágenes con nombres únicos

while True:
    ret, frame = cap.read()  # Captura el frame actual
    if not ret:
        break

    # Guardar la imagen original capturada
    cv2.imwrite(f"{output_dir}/original_{image_count}.jpg", frame)

    # Convertir la imagen a escala de grises, pero no reducir la resolución aún
    img_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Redimensionar la imagen a 48x48 directamente con una interpolación adecuada (INTER_AREA)
    img_gray_resized = cv2.resize(img_gray, (48, 48), interpolation=cv2.INTER_AREA)  # Evitar pérdida de calidad

    # Guardar la imagen redimensionada a 48x48 para inspección
    cv2.imwrite(f"{output_dir}/preprocessed_{image_count}.jpg", img_gray_resized)

    # Convertir la imagen redimensionada a tensor para el modelo
    img_tensor = transform(Image.fromarray(img_gray_resized)).unsqueeze(0).to(device)

    # Realiza la predicción
    with torch.no_grad():
        output = model(img_tensor)
        _, predicted = torch.max(output, 1)
        emotion = emotion_map[predicted.item()]

    # Muestra la predicción en el frame
    cv2.putText(frame, f'Emocion: {emotion}', (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # Muestra el frame con la predicción
    cv2.imshow('Reconocimiento de Emociones en Tiempo Real', frame)

    # Incrementar el contador de imágenes
    image_count += 1

    # Presiona 'q' para salir del bucle
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Libera los recursos
cap.release()
cv2.destroyAllWindows()


/var/folders/pv/hj1tkdtn4xd2lk842kp9xy7r0000gn/T/ipykernel_5972/2514918474.py:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load('modelo_emociones_ima

KeyboardInterrupt: 

In [3]:
import torch
import torch.nn as nn
from torchvision import models, transforms
import cv2
from PIL import Image
import numpy as np
import os

# Inicializar el modelo
model = models.resnet18(pretrained=False)
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
model.fc = nn.Linear(model.fc.in_features, 7)

# Cargar los pesos del modelo en la CPU
state_dict = torch.load('modelo_emociones_imagen.pth', map_location=torch.device('cpu'))
model.load_state_dict(state_dict)

# Asegurarse de que el modelo esté en modo de evaluación y en la CPU
device = torch.device('cpu')
model = model.to(device)
model.eval()  # Asegurarse de que el modelo está en modo de evaluación

# Define las transformaciones necesarias
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Mapeo de índices a nombres de emociones
emotion_map = {0: 'Enojo', 1: 'Disgusto', 2: 'Miedo', 3: 'Felicidad', 4: 'Tristeza', 5: 'Sorpresa', 6: 'Neutral'}

# Crear una carpeta para guardar las imágenes si no existe
output_dir = "captured_images"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Captura de video desde la cámara
cap = cv2.VideoCapture(0)  # 0 para la cámara por defecto

image_count = 0  # Contador para guardar imágenes con nombres únicos

while True:
    ret, frame = cap.read()  # Captura el frame actual
    if not ret:
        break

    # Guardar la imagen original capturada
    cv2.imwrite(f"{output_dir}/original_{image_count}.jpg", frame)

    # Convertir la imagen a escala de grises
    img_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Redimensionar la imagen a 48x48 usando interpolación cúbica para mayor calidad
    img_gray_resized = cv2.resize(img_gray, (48, 48), interpolation=cv2.INTER_CUBIC)

    # Guardar la imagen redimensionada a 48x48 para inspección
    cv2.imwrite(f"{output_dir}/preprocessed_{image_count}.jpg", img_gray_resized)

    # Convertir la imagen redimensionada a tensor para el modelo
    img_tensor = transform(Image.fromarray(img_gray_resized)).unsqueeze(0).to(device)

    # Realiza la predicción
    with torch.no_grad():
        output = model(img_tensor)
        _, predicted = torch.max(output, 1)
        emotion = emotion_map[predicted.item()]

    # Muestra la predicción en el frame
    cv2.putText(frame, f'Emocion: {emotion}', (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # Muestra el frame con la predicc


/var/folders/pv/hj1tkdtn4xd2lk842kp9xy7r0000gn/T/ipykernel_6605/439842556.py:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load('modelo_emociones_imag

KeyboardInterrupt: 

In [2]:
# Release resources
cap.release()

# Destroy all OpenCV windows
cv2.destroyAllWindows()
